In [259]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [260]:
# Set path
path = r'/Users/andrewfearney27/Instacart Basket Analysis'
file_path = os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_combined.csv')


In [261]:
# Load the DataFrame from the CSV file
ords_prods_merge = pd.read_csv(file_path)


In [262]:
# Display the first few rows of the DataFrame to confirm loading
print(ords_prods_merge.head())

   order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2539329        1    prior             1          2                  8   
1   2539329        1    prior             1          2                  8   
2   2539329        1    prior             1          2                  8   
3   2539329        1    prior             1          2                  8   
4   2539329        1    prior             1          2                  8   

   days_since_prior_order  product_id  add_to_cart_order  reordered _merge  
0                     NaN         196                  1          0   both  
1                     NaN       14084                  2          0   both  
2                     NaN       12427                  3          0   both  
3                     NaN       26088                  4          0   both  
4                     NaN       26405                  5          0   both  


In [263]:
# Remove the 'eval_set' column if it exists
if 'eval_set' in ords_prods_merge.columns:
    ords_prods_merge = ords_prods_merge.drop(columns=['eval_set'])
    print("eval_set column dropped.")
else:
    print("eval_set column does not exist.")

eval_set column dropped.


In [264]:
# Create a subset
df = ords_prods_merge[:1000000]

In [265]:
df.shape

(1000000, 10)

In [266]:
df.head(10)

order_id  user_id  order_number  order_dow  order_hour_of_day  \
0   2539329        1             1          2                  8   
1   2539329        1             1          2                  8   
2   2539329        1             1          2                  8   
3   2539329        1             1          2                  8   
4   2539329        1             1          2                  8   
5   2398795        1             2          3                  7   
6   2398795        1             2          3                  7   
7   2398795        1             2          3                  7   
8   2398795        1             2          3                  7   
9   2398795        1             2          3                  7   

   days_since_prior_order  product_id  add_to_cart_order  reordered _merge  
0                     NaN         196                  1          0   both  
1                     NaN       14084                  2          0   both  
2                     NaN       12427                  3          0   both  
3                     NaN       26088                  4          0   both  
4                     NaN       26405                  5          0   both  
5                    15.0         196                  1          1   both  
6                    15.0       10258                  2          0   both  
7                    15.0       12427                  3          1   both  
8                    15.0       13176                  4          0   both  
9                    15.0       26088                  5          1   both

In [267]:
# Calculate the mean of order_number for the entire DataFrame
mean_order_number_entire = ords_prods_merge.groupby('product_id')['order_number'].agg(['mean'])
print(mean_order_number_entire)


                 mean
product_id           
1           19.397408
2           25.666667
3           17.342960
4            9.027356
5            8.933333
...               ...
49684        4.777778
49685       13.448980
49686       19.316667
49687       13.846154
49688        7.629213

[49677 rows x 1 columns]


In [268]:
#3 Analysis
#Range of Mean Orders: The mean order numbers for different departments vary significantly, ranging from approximately 8.9 (department_id 5) to 22.9 (department_id 21).

#Comparison with Subset: The mean order numbers for the entire dataframe are generally higher compared to the subset. This suggests that when considering the full dataset, users tend to place more orders across departments.

In [269]:
#4 Create a flag loyalty 
#Create a max order column
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)
print(ords_prods_merge.head(15))

/var/folders/mh/jgcdcff53n3ft01xcw7qxswc0000gn/T/ipykernel_47328/3567946116.py:3: FutureWarning: The provided callable <function max at 0x108f17920> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


    order_id  user_id  order_number  order_dow  order_hour_of_day  \
0    2539329        1             1          2                  8   
1    2539329        1             1          2                  8   
2    2539329        1             1          2                  8   
3    2539329        1             1          2                  8   
4    2539329        1             1          2                  8   
5    2398795        1             2          3                  7   
6    2398795        1             2          3                  7   
7    2398795        1             2          3                  7   
8    2398795        1             2          3                  7   
9    2398795        1             2          3                  7   
10   2398795        1             2          3                  7   
11    473747        1             3          3                 12   
12    473747        1             3          3                 12   
13    473747        1             

In [270]:
# Create an empty 'loyalty_flag' column
ords_prods_merge['loyalty_flag'] = np.nan


In [271]:
#Define the categories for the 'loyalty_flag' column
categories = ['New customer', 'Regular customer', 'Loyal customer']
ords_prods_merge['loyalty_flag'] = pd.Categorical(ords_prods_merge['loyalty_flag'], categories=categories)


In [272]:
#Create a flag loyalty for existing customers
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [273]:
print(ords_prods_merge[['user_id', 'order_number', 'loyalty_flag']].head(15))


    user_id  order_number  loyalty_flag
0         1             1  New customer
1         1             1  New customer
2         1             1  New customer
3         1             1  New customer
4         1             1  New customer
5         1             2  New customer
6         1             2  New customer
7         1             2  New customer
8         1             2  New customer
9         1             2  New customer
10        1             2  New customer
11        1             3  New customer
12        1             3  New customer
13        1             3  New customer
14        1             3  New customer


In [274]:
# Assuming you have a DataFrame 'products' that contains 'product_id' and 'price'
products = pd.read_csv('path_to_products_data.csv') # Ensure the correct path

# Merge the products DataFrame with the existing orders DataFrame
ords_prods_merge = pd.merge(ords_prods_merge, products[['product_id', 'price']], on='product_id', how='left')

# Rename 'price' column to 'prices' if needed
ords_prods_merge.rename(columns={'price': 'prices'}, inplace=True)

# Verify the merge
print(ords_prods_merge.head())


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_products_data.csv'

In [ ]:
# Now to calculate mean order number for each user
ords_prods_merge['mean_order_number'] = ords_prods_merge.groupby('user_id')['order_number'].transform('mean')

# Verify the 'mean_order_number' column
print(ords_prods_merge[['user_id', 'mean_order_number']].head())

#Group by 'loyalty_flag' and Aggregate
spending_stats = ords_prods_merge.groupby('loyalty_flag')['mean_order_number'].agg(['mean', 'min', 'max'])
print(spending_stats)

# Analysis of Spending Habits
# Based on the calculated statistics, we can see the following spending habits for each loyalty group:
# Loyal Customers: On average, they have around `mean` orders, with the lowest being `min` and the highest being `max`.
# Regular Customers: Their average number of orders is `mean`, with a range between `min` and `max`.
# New Customers: They have an average number of orders of `mean`, with the minimum being `min` and the maximum being `max`.


In [285]:
#6 Determine order frequency of each user

In [286]:
#Calculate the mean order number for each user
ords_prods_merge['mean_order_number'] = ords_prods_merge.groupby('user_id')['order_number'].transform('mean')

In [287]:
#Verify the 'mean_order_number' column
print(ords_prods_merge[['user_id', 'mean_order_number']].head())

   user_id  mean_order_number
0        1           5.813559
1        1           5.813559
2        1           5.813559
3        1           5.813559
4        1           5.813559


In [288]:
#Create spending_flag column with given criteria using loc() function
ords_prods_merge.loc[ords_prods_merge['mean_order_number'] >= 10, 'spending_flag'] = 'High spender'
ords_prods_merge.loc[ords_prods_merge['mean_order_number'] < 10, 'spending_flag'] = 'Low spender'

In [289]:
#Check frequency of new spending_flag column
print(ords_prods_merge['spending_flag'].value_counts(dropna=False))

spending_flag
High spender    20350720
Low spender     12083769
Name: count, dtype: int64


In [290]:
#View relevant columns of updated data frame
print(ords_prods_merge[['user_id', 'mean_order_number', 'spending_flag']].head(20))


    user_id  mean_order_number spending_flag
0         1           5.813559   Low spender
1         1           5.813559   Low spender
2         1           5.813559   Low spender
3         1           5.813559   Low spender
4         1           5.813559   Low spender
5         1           5.813559   Low spender
6         1           5.813559   Low spender
7         1           5.813559   Low spender
8         1           5.813559   Low spender
9         1           5.813559   Low spender
10        1           5.813559   Low spender
11        1           5.813559   Low spender
12        1           5.813559   Low spender
13        1           5.813559   Low spender
14        1           5.813559   Low spender
15        1           5.813559   Low spender
16        1           5.813559   Low spender
17        1           5.813559   Low spender
18        1           5.813559   Low spender
19        1           5.813559   Low spender


In [291]:
#7 Determining order frequency of each user

In [292]:
ords_prods_merge['median_days_between_orders'] = ords_prods_merge.groupby('user_id')['days_since_prior_order'].transform(np.median)


/var/folders/mh/jgcdcff53n3ft01xcw7qxswc0000gn/T/ipykernel_47328/1000495780.py:1: FutureWarning: The provided callable <function median at 0x109060400> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['median_days_between_orders'] = ords_prods_merge.groupby('user_id')['days_since_prior_order'].transform(np.median)


In [293]:
#Verify the 'median_days_between_orders' column
print(ords_prods_merge[['user_id', 'days_since_prior_order', 'median_days_between_orders']].head(20))


    user_id  days_since_prior_order  median_days_between_orders
0         1                     NaN                        20.5
1         1                     NaN                        20.5
2         1                     NaN                        20.5
3         1                     NaN                        20.5
4         1                     NaN                        20.5
5         1                    15.0                        20.5
6         1                    15.0                        20.5
7         1                    15.0                        20.5
8         1                    15.0                        20.5
9         1                    15.0                        20.5
10        1                    15.0                        20.5
11        1                    21.0                        20.5
12        1                    21.0                        20.5
13        1                    21.0                        20.5
14        1                    21.0     

In [294]:
#Create an order frequency flag based on the given criteria using the loc() function
ords_prods_merge.loc[ords_prods_merge['median_days_between_orders'] > 20, 'order_frequency'] = 'Non-frequent customer'
ords_prods_merge.loc[(ords_prods_merge['median_days_between_orders'] <= 20) & (ords_prods_merge['median_days_between_orders'] > 10), 'order_frequency'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['median_days_between_orders'] <= 10, 'order_frequency'] = 'Frequent customer'


In [295]:
#Check value counts of the new 'order_frequency' column
print(ords_prods_merge['order_frequency'].value_counts(dropna=False))


order_frequency
Frequent customer        21577653
Regular customer          7217190
Non-frequent customer     3639646
Name: count, dtype: int64


In [300]:
print(ords_prods_merge[['user_id', 'days_since_prior_order', 'median_days_between_orders', 'order_frequency']].head(25))


    user_id  days_since_prior_order  median_days_between_orders  \
0         1                     NaN                        20.5   
1         1                     NaN                        20.5   
2         1                     NaN                        20.5   
3         1                     NaN                        20.5   
4         1                     NaN                        20.5   
5         1                    15.0                        20.5   
6         1                    15.0                        20.5   
7         1                    15.0                        20.5   
8         1                    15.0                        20.5   
9         1                    15.0                        20.5   
10        1                    15.0                        20.5   
11        1                    21.0                        20.5   
12        1                    21.0                        20.5   
13        1                    21.0                        20.

In [305]:
#7 Exporting dataframe

In [306]:
# Define the path where the DataFrame will be saved
path = 'Instacart Basket Analysis/02 Data/Prepared Data/'
filename = 'ords_prods_grouped.pkl'


In [307]:
# Ensure the directory exists
os.makedirs(path, exist_ok=True)


In [308]:
# Export the DataFrame as a pickle file
ords_prods_merge.to_pickle(os.path.join(path, filename))

print("DataFrame successfully exported to", os.path.join(path, filename))

DataFrame successfully exported to Instacart Basket Analysis/02 Data/Prepared Data/ords_prods_grouped.pkl
